In [1]:
import os
import pandas as pd
import folium
import numpy as np
import geopandas as gpd

In [2]:
date = '2021.10.20'

In [3]:
dir_main = os.getcwd()
dir_data = os.path.join(dir_main, 'data')
dir_maps = os.path.join(dir_main, 'maps')

In [4]:
lst_names = ['vehicle.csv', 'race.csv']

In [5]:
lst_dfs = []
for name in lst_names:
    df_temp = pd.read_csv(os.path.join(dir_data, name), header=None)
    
    df_temp.drop(df_temp.index[0], inplace=True) # Delete unneeded header
    new_header = df_temp.iloc[0] # Grab the first row for the header
    df_temp = df_temp[1:] # Take the data less the header row
    df_temp.columns = new_header # Set the header row as the df header
    
    lst_dfs.append(df_temp)

In [6]:
from functools import reduce

In [7]:
df_merged = reduce(lambda x, y: pd.merge(x,y, on='id'), lst_dfs)

In [8]:
df_stats = df_merged[['id',
                      'Total Pop',
                      'Black Percent',
                      'No Vehicle Household Percent']]

In [9]:
week = date
df_vac = pd.read_csv(os.path.join(dir_main, 'data', f'{week}_Vaccinations by Tract.csv'))

In [10]:
# Add full tract code to vaccination tracts
tracts = df_vac['Tract ID']
geoid = []
for tract in tracts:
    geoid.append('1400000US' + str(tract))
df_vac['id']  = geoid  

In [11]:
df_vac = df_vac[['id', 'Parish', 'x', 'y', 'Series Intitiated', 'Series Completed']]

In [12]:
df = df_vac.merge(df_stats, on='id', copy=False)

In [13]:
df = df[df.Parish == 'Orleans'] # Limit to only Orleans Parish

In [14]:
# Convert columns to float
# The linear modeling will 
column_temp = df['id']
df.drop(columns=['id', 'Parish'], inplace=True)
df = df.apply(pd.to_numeric, errors='coerce') # Convert all columns to numeric
df['id'] = column_temp

df.dtypes

x                               float64
y                               float64
Series Intitiated               float64
Series Completed                float64
Total Pop                         int64
Black Percent                   float64
No Vehicle Household Percent    float64
id                               object
dtype: object

In [15]:
# Limit to only tracts with > 50% Black population
df = df[df['Black Percent'] > 50]

In [16]:
# Calculate percent of census tract pop completely vaccinated
df['SeriesComp Percent'] = df['Series Completed'] / df['Total Pop']

In [17]:
df['weighted'] = df['SeriesComp Percent'] * df['No Vehicle Household Percent']

In [18]:
df['SeriesComp Inverse'] = 1/df['SeriesComp Percent']
df['SeriesComp Percent'] = round(df['SeriesComp Percent']*100, 2)
df['Black Percent'] = round(df['Black Percent'], 2)

In [19]:
df

,x,y,Series Intitiated,Series Completed,Total Pop,Black Percent,No Vehicle Household Percent,id,SeriesComp Percent,weighted,SeriesComp Inverse
615,-90.052353,29.945894,636.0,555.0,1202,72.88,25.364964,1400000US22071000200,46.17,11.711776,2.165766
616,-90.048988,29.940950,648.0,570.0,1361,90.23,33.059548,1400000US22071000300,41.88,13.845659,2.387719
617,-90.042480,29.946333,1252.0,1094.0,1937,78.58,22.180917,1400000US22071000400,56.48,12.527580,1.770567
618,-90.040955,29.932783,448.0,378.0,953,97.48,30.174564,1400000US22071000601,39.66,11.968505,2.521164
619,-90.033455,29.937494,1336.0,1149.0,2364,80.25,21.564482,1400000US22071000602,48.60,10.481214,2.057441
...,...,...,...,...,...,...,...,...,...,...,...
780,-90.079575,29.996105,1475.0,1306.0,2664,90.80,28.078358,1400000US22071013800,49.02,13.765141,2.039816
782,-90.083099,29.944670,913.0,780.0,2054,92.70,57.760532,1400000US22071014000,37.97,21.934379,2.633333
783,-90.070679,29.925150,1355.0,1199.0,2369,77.33,43.442623,1400000US22071014100,50.61,21.987212,1.975813
785,-90.092499,29.939842,1016.0,851.0,1809,92.59,54.498044,1400000US22071014300,47.04,25.637278,2.125734


In [20]:
df_dropped = df.dropna()

In [21]:
gdf_tracts = gpd.read_file(os.path.join(dir_data, 'tiger_tracts_clean.geojson'))

In [22]:
gdf_tracts['id'] = gdf_tracts['GEO_ID']

In [23]:
gdf = gdf_tracts.merge(df_dropped, on='id')

In [24]:
from statistics import mean

In [25]:
mean_lat = mean(df_dropped['y'])
mean_lon = mean(df_dropped['x'])
mean_lon

-90.04395606185564

In [26]:
gdf

,GEO_ID,geometry,id,x,y,Series Intitiated,Series Completed,Total Pop,Black Percent,No Vehicle Household Percent,SeriesComp Percent,weighted,SeriesComp Inverse
0,1400000US22071000200,"POLYGON ((-90.05857 29.94650, -90.05855 29.945...",1400000US22071000200,-90.052353,29.945894,636.0,555.0,1202,72.88,25.364964,46.17,11.711776,2.165766
1,1400000US22071000300,"POLYGON ((-90.05406 29.94097, -90.05413 29.940...",1400000US22071000300,-90.048988,29.940950,648.0,570.0,1361,90.23,33.059548,41.88,13.845659,2.387719
2,1400000US22071000400,"POLYGON ((-90.04779 29.93482, -90.04677 29.933...",1400000US22071000400,-90.042480,29.946333,1252.0,1094.0,1937,78.58,22.180917,56.48,12.527580,1.770567
3,1400000US22071000601,"POLYGON ((-90.04397 29.93141, -90.04397 29.931...",1400000US22071000601,-90.040955,29.932783,448.0,378.0,953,97.48,30.174564,39.66,11.968505,2.521164
4,1400000US22071000602,"POLYGON ((-90.04122 29.93760, -90.04076 29.937...",1400000US22071000602,-90.033455,29.937494,1336.0,1149.0,2364,80.25,21.564482,48.60,10.481214,2.057441
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1400000US22071013700,"POLYGON ((-90.04586 30.00274, -90.04585 30.002...",1400000US22071013700,-90.033051,29.993980,1423.0,1178.0,2427,98.19,38.049941,48.54,18.468410,2.060272
93,1400000US22071013800,"POLYGON ((-90.08628 29.99275, -90.08629 29.992...",1400000US22071013800,-90.079575,29.996105,1475.0,1306.0,2664,90.80,28.078358,49.02,13.765141,2.039816
94,1400000US22071014000,"POLYGON ((-90.08850 29.94599, -90.08837 29.945...",1400000US22071014000,-90.083099,29.944670,913.0,780.0,2054,92.70,57.760532,37.97,21.934379,2.633333
95,1400000US22071014100,"POLYGON ((-90.07608 29.92381, -90.07573 29.923...",1400000US22071014100,-90.070679,29.925150,1355.0,1199.0,2369,77.33,43.442623,50.61,21.987212,1.975813


In [27]:
m = folium.Map(location=[mean_lat, -90],
               zoom_start=12,
               tiles=None)
folium.raster_layers.TileLayer(tiles='Stamen Toner', 
                               min_zoom=0, max_zoom=18, 
                               max_native_zoom=None, attr=None, 
                               API_key=None, detect_retina=False, 
                               name=None, overlay=False, control=False, 
                               show=True, no_wrap=False, subdomains='abc', 
                               tms=False, opacity=0.3
                              ).add_to(m)

folium.Choropleth(geo_data=gdf,
                  name='Weighted TSYI',
                  data=gdf,
                  columns=['id', 'SeriesComp Percent'],
                  key_on='feature.properties.id',
                  bins=9,
                  fill_color='YlOrRd_r',
                  nan_fill_color='black',
                  fill_opacity=0.7,
                  nan_fill_opacity=0,
                  line_color='black',
                  line_weight=0.5,
                  line_opacity=0.4,
                  legend_name=f'Percent Vaccinated of Total Population',
                  highlight=True,
                  smooth_factor=1).add_to(m)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#6f6f6f', 
                            'fillOpacity': 0, 
                            'weight': 0.1
                           }
highlight_function = lambda x: {#'fillColor': '#f3e20c',
                                'color':'white',
                                'fillOpacity': 0.0,
                                'weight': 5
                               }
    
popup = folium.features.GeoJson(
    gdf,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(fields=['SeriesComp Percent',
                                                   'Black Percent'],
                                           aliases=['Percent Fully Vaccinated of Total Population:',
                                                    'Percent Black Population: '                                                   ],
                                           style=("background-color: grey; color: #ffffff; font-family: arial; font-size: 12px; padding: 10px;")
                                          )
)
m.add_child(popup)
m.keep_in_front(popup)

map_name = f'map_{week}.html'
m.save(os.path.join(dir_maps, map_name))
map_current = m.save('map_current.html')

In [28]:
# Write out JS file with the date for HTML

with open('date.js', 'w') as f:
    f.write(f'document.getElementById("mapWeek").innerHTML = "Updated: {week}";')

In [29]:
# Upload newest map and date.js to server
import ftplib
session = ftplib.FTP('****website*****', 
                     '****login*****',
                     '****password*****')

server_vac_dir = '/abnormaldistributions.com/public_html/tsyi/vac_rates'

with open('map_current.html', 'rb') as map_cur:
    session.cwd(server_vac_dir)
    session.storbinary('STOR map_current.html', map_cur)
with open('date.js', 'rb') as js_date:
    session.cwd(server_vac_dir)
    session.storbinary('STOR date.js', js_date)
session.quit()

'221-Goodbye. You uploaded 467 and downloaded 0 kbytes.\n221 Logout.'

In [30]:
m